# Load multiple COGs into xarray dataset

In [11]:
import satsearch
from satstac import Items
import geopandas as gpd
import xarray as xr

In [2]:
gfa = gpd.read_file('aoi.geojson')
gfa

,geometry
0,"POLYGON ((-123.3 46.27103747280261, -122.16796..."


In [3]:
bbox = gfa.bounds.values[0]
west, north, east, south = bbox

In [4]:
properties =  ["landsat:tier=T1"] 

bbox = (west, south, east, north) #(min lon, min lat, max lon, max lat)

results = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        datetime='2019-07-15/2019-07-31',
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())

4 items


In [5]:
results.found()

4

In [6]:
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv

In [7]:
items = results.items()
items.save('items-landsat8.json')
items = Items.load('items-landsat8.json')
#items.bbox()

In [8]:
gfl = gpd.read_file('items-landsat8.json')
gfl = gfl.sort_values('datetime').reset_index(drop=True)
print('records:', len(gfl))
gfl.head()

records: 4


,id,collection,eo:gsd,eo:platform,eo:instrument,eo:off_nadir,eo:bands,datetime,eo:sun_azimuth,eo:sun_elevation,eo:cloud_cover,eo:row,eo:column,landsat:product_id,landsat:scene_id,landsat:processing_level,landsat:tier,landsat:revision,eo:epsg,geometry
0,LC80460272019201,landsat-8-l1,15,landsat-8,OLI_TIRS,0,"[ { ""full_width_half_max"": 0.02, ""center_wavel...",2019-07-20T18:55:37,142.841478,58.863403,1,027,046,LC08_L1TP_046027_20190720_20190731_01_T1,LC80460272019201LGN00,L1TP,T1,00,32610,POLYGON ((-122.7195543570606 48.51225662720523...
1,LC80460282019201,landsat-8-l1,15,landsat-8,OLI_TIRS,0,"[ { ""full_width_half_max"": 0.02, ""center_wavel...",2019-07-20T18:56:01,140.640001,59.791567,0,028,046,LC08_L1TP_046028_20190720_20190731_01_T1,LC80460282019201LGN00,L1TP,T1,00,32610,POLYGON ((-123.2399397415835 47.08783413818369...
2,LC80470272019208,landsat-8-l1,15,landsat-8,OLI_TIRS,0,"[ { ""full_width_half_max"": 0.02, ""center_wavel...",2019-07-27T19:01:51,143.987513,57.532989,76,027,047,LC08_L1TP_047027_20190727_20190801_01_T1,LC80470272019208LGN00,L1TP,T1,00,32610,POLYGON ((-124.2666148235796 48.51254572942294...
3,LC80470282019208,landsat-8-l1,15,landsat-8,OLI_TIRS,0,"[ { ""full_width_half_max"": 0.02, ""center_wavel...",2019-07-27T19:02:15,141.913254,58.484393,52,028,047,LC08_L1TP_047028_20190727_20190801_01_T1,LC80470282019208LGN00,L1TP,T1,00,32610,POLYGON ((-124.7869013469721 47.10292200101978...


In [9]:
# Plot search AOI and frames on a map

# just keep id for hover tips
cols = gfl.loc[:,('id','geometry')]

footprints = cols.hvplot(geo=True, line_color='k', alpha=0.1, title='Landsat 8 T1')
aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
tiles = gv.tile_sources.CartoEco.options(width=700, height=500) #.redim.range(Latitude=(45, 50), Longitude=(-126,-120)) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * aoi * labels

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Polygons.I  :Polygons   [Longitude,Latitude]   (id)
   .Polygons.II :Polygons   [Longitude,Latitude]
   .WMTS.II     :WMTS   [Longitude,Latitude]

In [25]:
# Print URLS's we'll work with
[print(items[x].asset('nir')['href']) for x in range(4)]

https://landsat-pds.s3.amazonaws.com/c1/L8/046/027/LC08_L1TP_046027_20190720_20190731_01_T1/LC08_L1TP_046027_20190720_20190731_01_T1_B5.TIF
https://landsat-pds.s3.amazonaws.com/c1/L8/046/028/LC08_L1TP_046028_20190720_20190731_01_T1/LC08_L1TP_046028_20190720_20190731_01_T1_B5.TIF
https://landsat-pds.s3.amazonaws.com/c1/L8/047/027/LC08_L1TP_047027_20190727_20190801_01_T1/LC08_L1TP_047027_20190727_20190801_01_T1_B5.TIF
https://landsat-pds.s3.amazonaws.com/c1/L8/047/028/LC08_L1TP_047028_20190727_20190801_01_T1/LC08_L1TP_047028_20190727_20190801_01_T1_B5.TIF


[None, None, None, None]

In [31]:
# Load these into an xarray dataset
# NIR band URLS of these 4 images
da1 = xr.open_rasterio(items[0].asset('nir')['href'], chunks={'band': 1, 'x': 1024, 'y': 1024})
da2 = xr.open_rasterio(items[1].asset('nir')['href'], chunks={'band': 1, 'x': 1024, 'y': 1024})
da3 = xr.open_rasterio(items[2].asset('nir')['href'], chunks={'band': 1, 'x': 1024, 'y': 1024})
da4 = xr.open_rasterio(items[3].asset('nir')['href'], chunks={'band': 1, 'x': 1024, 'y': 1024})

In [55]:
# NOTE: per julia signells suggestino, want filename in attributes as well
da1.attrs['filename'] = items[0].id
da1.attrs['date'] = items[0].date.strftime('%Y-%m-%d')
da1

<xarray.DataArray (band: 1, y: 7901, x: 7781)>
dask.array<shape=(1, 7901, 7781), dtype=uint16, chunksize=(1, 1024, 1024)>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 5.216e+06 5.215e+06 5.215e+06 ... 4.979e+06 4.978e+06
  * x        (x) float64 4.344e+05 4.344e+05 4.345e+05 ... 6.678e+05 6.678e+05
Attributes:
    transform:   (30.0, 0.0, 434385.0, 0.0, -30.0, 5215515.0)
    crs:         +init=epsg:32610
    res:         (30.0, 30.0)
    is_tiled:    1
    nodatavals:  (nan,)
    scales:      (1.0,)
    offsets:     (0.0,)
    filename:    LC80460272019201
    date:        2019-07-20

In [33]:
da2

<xarray.DataArray (band: 1, y: 7901, x: 7781)>
dask.array<shape=(1, 7901, 7781), dtype=uint16, chunksize=(1, 1024, 1024)>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 5.216e+06 5.215e+06 5.215e+06 ... 4.979e+06 4.978e+06
  * x        (x) float64 4.344e+05 4.344e+05 4.345e+05 ... 6.678e+05 6.678e+05
Attributes:
    transform:   (30.0, 0.0, 434385.0, 0.0, -30.0, 5215515.0)
    crs:         +init=epsg:32610
    res:         (30.0, 30.0)
    is_tiled:    1
    nodatavals:  (nan,)
    scales:      (1.0,)
    offsets:     (0.0,)

In [35]:
da3

<xarray.DataArray (band: 1, y: 7971, x: 7861)>
dask.array<shape=(1, 7971, 7861), dtype=uint16, chunksize=(1, 1024, 1024)>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 5.374e+06 5.374e+06 5.374e+06 ... 5.135e+06 5.135e+06
  * x        (x) float64 3.558e+05 3.558e+05 3.559e+05 ... 5.916e+05 5.916e+05
Attributes:
    transform:   (30.0, 0.0, 355785.0, 0.0, -30.0, 5374215.0)
    crs:         +init=epsg:32610
    res:         (30.0, 30.0)
    is_tiled:    1
    nodatavals:  (nan,)
    scales:      (1.0,)
    offsets:     (0.0,)

In [36]:
da4

<xarray.DataArray (band: 1, y: 7981, x: 7871)>
dask.array<shape=(1, 7981, 7871), dtype=uint16, chunksize=(1, 1024, 1024)>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 5.217e+06 5.217e+06 5.217e+06 ... 4.978e+06 4.978e+06
  * x        (x) float64 3.132e+05 3.132e+05 3.133e+05 ... 5.493e+05 5.493e+05
Attributes:
    transform:   (30.0, 0.0, 313185.0, 0.0, -30.0, 5217015.0)
    crs:         +init=epsg:32610
    res:         (30.0, 30.0)
    is_tiled:    1
    nodatavals:  (nan,)
    scales:      (1.0,)
    offsets:     (0.0,)

In [37]:
# NOTE: these have the same CRS, but slightly different grids and dimensions

In [39]:
# Visualize 1
da1.hvplot.image(rasterize=True, width=700, height=500, cmap='magma')

/srv/conda/envs/notebook/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../srv/conda/envs/notebook/lib/python3.7/site-packages/datashader/resampling.py", line 235:
@ngjit_parallel
def _get_dimensions(src, out):
^

  self.func_ir.loc))


:DynamicMap   [band]
   :Image   [x,y]   (value)

In [40]:
#index = gpd.pd.DatetimeIndex(items.dates(), name='time')
datasets = [da1,da2,da3,da4]
da = xr.concat(datasets, dim=index)
print('Dataset size (Gb): ', da.nbytes/1e9)

NameError: name 'index' is not defined

odict_keys(['transform', 'crs', 'res', 'is_tiled', 'nodatavals', 'scales', 'offsets'])